In [18]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

from sklearn import svm
from decimal import *

read train,val and test from files

In [5]:
train = pd.read_csv('input/train.tsv', sep='\t')
test = pd.read_csv('input/test.tsv', sep='\t')

In [6]:
X = train.iloc[:100,1:151]

In [7]:
y = train.loc[:99,"target"]

In [14]:
kfold = KFold(10)

In [20]:
for train_index , test_index in kfold.split(X):
    zero_features = []
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
    mse,zero_features = (Lasso_finetuning(X_train,y_train,X_test,y_test))
    #lasso_dict[mse] = zero_features

    # drop zero corelation features
    X_train.drop(zero_features, axis = 1, inplace=True)
    X_test.drop(zero_features, axis = 1, inplace=True)

    #svm classifier
    #clf = svm.SVC(kernel='linear')
    #clf.fit(X_train,y_train)
    #y_pred = clf.predict(X_test)
    #print("Accuracy: %.3f" % metrics.accuracy_score(y_test, y_pred))
    
    forest = RandomForestClassifier(criterion='gini',
                                 n_estimators=5,
                                 random_state=1,
                                 n_jobs=2)
    
    forest.fit(X_train, y_train)

    y_pred = forest.predict(X_test)
    print('Accuracy: %.3f' % metrics.accuracy_score(y_test, y_pred))
    
    
    break
    
    

c:\users\giankond\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,
c:\users\giankond\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler 

Accuracy: 0.700


**Lasso_finetuning** : returns a pair of mse of train dataset and zero corelation features

In [16]:
def Lasso_finetuning(X_train, y_train,X_test,Y_test):
    lasso = Lasso(max_iter=10000, normalize=True)
    #alphas = [0.0001, 0.00001, 0.001, 0.01, 0.1]
    lassocv = LassoCV(alphas=None, cv=50, max_iter=100000, normalize=True)
    lassocv.fit(X_train, y_train)
    lasso.set_params(alpha=lassocv.alpha_)
    lasso.fit(X_train, y_train)
    mse = mean_squared_error(y_test, lasso.predict(X_test))

    #print("alpha : ",lassocv.alpha_)
    zero_features = []
    zero_count = 0;
    ser = pd.Series(lasso.coef_, index=X_train.columns)
    for i, v in ser.items():
        #print(i,v)
        if (abs(v) == 0):
            zero_count += 1
            zero_features.append(i)
    #print(zero_count,' zero corelation features')
    return mse,zero_features